# Analyze the cProfile output

We will use the `pstats` module to read and analyze the outupt of the profiler.


In [23]:
import pstats
mystats = pstats.Stats("profiler_output.txt")

## Display the top 15 functions that took the longest time to run

Even in such a short script, python called millions of function! A lot of them are super quick and really not what we are after. Typically, we want to find the few functions that are bottlenecks in our code.

The function below sorts the functions by which took the longest to run, and prints out only the stats of the top 15 functions that took longest to run. The columns below mean:
  * ncalls: number of times this function was called
  * tottime: the total execution time spent in this code NOT including calls to other functions
  * percall: this first percall divides tottime by ncalls. The amount of time per call spent solely in this function.
  * cumtime: the total execution time spent in this code INCLUDING calls to sub functions. `cumtime > tottime always.
  * percall: second percall divides cumtime by ncalls
  * filename: the name of the function being considered

Both `tottime` and `cumtime` are useful. A function with a high `tottime` means we should focus on speeding up this function. A function with only a high `cumtime` means we should see what this function is calling to improve runtime.
  * `_newton_solver()` is a function with a higher tottime. We should look at the lines in that function to check out how to speed it up.
  * `_logl()` is a function with a low tottime but a very high `cumtime`. We should look at the functions it calls to figure out what takes up so much time.

Generally, we want to put some filters on `print_stats`, becuase otherwise there will be so much printed out it is unmanageable. 


In [11]:
mystats.sort_stats("time").print_stats(15)

Thu Jun 11 18:14:25 2020    profiler_output.txt

         3100667 function calls (3058377 primitive calls) in 68.937 seconds

   Ordered by: internal time
   List reduced from 5237 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      774   17.867    0.023   19.654    0.025 c:\users\jason\github\orbitize\orbitize\kepler.py:173(_newton_solver)
      774   14.742    0.019   44.324    0.057 c:\users\jason\github\orbitize\orbitize\kepler.py:17(calc_orbit)
      387    7.687    0.020    7.769    0.020 c:\users\jason\github\orbitize\orbitize\lnlike.py:8(chi2_lnlike)
      774    3.166    0.004    3.166    0.004 c:\users\jason\github\orbitize\orbitize\system.py:379(radec2seppa)
      774    2.639    0.003   24.979    0.032 c:\users\jason\github\orbitize\orbitize\kepler.py:111(_calc_ecc_anom)
24227/23171    2.501    0.000    3.992    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    13946    1.919    0.000  

## Investigate why `_logl` takes so long

The `_logl()` function itself has a short runtime but it is calling something that takes a long time that makes it have a long `cumtime`. We can use the `print_callees()` function to look at the stats of all the functions it calls.

We can see that `compute_model` is the function with the highest cumtime, but its tottime is low, so we something it calls takes all the time. We must dig deeper!

In [24]:
mystats.print_callees('_logl')

Ordered by: call count
   List reduced from 5237 to 1 due to restriction <'_logl'>

Function                                                      called...
                                                                  ncalls  tottime  cumtime
c:\users\jason\github\orbitize\orbitize\sampler.py:49(_logl)  ->     387    0.003    0.979  <__array_function__ internals>:2(nansum)
                                                                    1548    0.006    0.011  C:\Users\Jason\Anaconda3\lib\site-packages\astropy\table\table.py:1600(__getitem__)
                                                                     387    7.687    7.769  c:\users\jason\github\orbitize\orbitize\lnlike.py:8(chi2_lnlike)
                                                                     387    1.328   41.250  c:\users\jason\github\orbitize\orbitize\system.py:184(compute_model)
                                                                     774    0.015    0.015  {built-in method numpy.array}




Let's dig deeper and check what slows down `compute_model`. 

It looks like it there's more of a rabbit hole. We can keep going, but let's stop here in the interest of time. 

In [25]:
mystats.sort_stats("time").print_callees('compute_model')


Ordered by: internal time
   List reduced from 5237 to 1 due to restriction <'compute_model'>

Function                                                              called...
                                                                          ncalls  tottime  cumtime
c:\users\jason\github\orbitize\orbitize\system.py:184(compute_model)  ->     387    0.003    0.005  C:\Users\Jason\Anaconda3\lib\site-packages\astropy\table\table.py:1600(__getitem__)
                                                                             387   12.711   37.174  c:\users\jason\github\orbitize\orbitize\kepler.py:17(calc_orbit)
                                                                             387    2.705    2.705  c:\users\jason\github\orbitize\orbitize\system.py:379(radec2seppa)
                                                                            3096    0.005    0.009  {built-in method builtins.len}
                                                                             38

We can also apply multiple filters. For example let's look at the top 15 numpy functions that took the most time to run. Note that the order of the filtering matters. This command first selects every function with numpy in the name, followed by taking the top 15. Calling `print_states(15, 'numpy')` would pick the top 15 longest runtime functions, and downselecting which has numpy in the name from those. That would give us less than 15 numpy functions.

In [26]:
mystats.sort_stats("time").print_stats('numpy', 15)

Thu Jun 11 18:14:25 2020    profiler_output.txt

         3100667 function calls (3058377 primitive calls) in 68.937 seconds

   Ordered by: internal time
   List reduced from 5237 to 483 due to restriction <'numpy'>
   List reduced from 483 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
24227/23171    2.501    0.000    3.992    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    31504    1.572    0.000    1.572    0.000 {built-in method numpy.array}
     2322    1.107    0.000    1.107    0.000 {method 'normal' of 'numpy.random.mtrand.RandomState' objects}
     2322    0.608    0.000    0.608    0.000 {method 'uniform' of 'numpy.random.mtrand.RandomState' objects}
      387    0.212    0.001    0.212    0.001 {method 'repeat' of 'numpy.ndarray' objects}
     1257    0.155    0.000    0.155    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      387    0.152    0.000    0.716    0.002 C:\Users\Jason

`numpy.array` gets called over 30,000 times. We can see which functions call it to look into potentially reducing the number of array creations to speed up the code. 

In [22]:
mystats.print_callers('numpy.array')

Ordered by: call count
   List reduced from 5237 to 1 due to restriction <'numpy.array'>

Function                       was called by...
                                   ncalls  tottime  cumtime
{built-in method numpy.array}  <-      77    0.000    0.000  C:\Users\Jason\Anaconda3\lib\site-packages\astropy\constants\constant.py:85(__new__)
                                        7    0.000    0.000  C:\Users\Jason\Anaconda3\lib\site-packages\astropy\io\ascii\core.py:841(convert_numpy)
                                       34    0.000    0.000  C:\Users\Jason\Anaconda3\lib\site-packages\astropy\table\column.py:218(__new__)
                                     5481    0.192    0.192  C:\Users\Jason\Anaconda3\lib\site-packages\astropy\units\quantity.py:290(__new__)
                                    16284    0.027    0.027  C:\Users\Jason\Anaconda3\lib\site-packages\astropy\units\quantity.py:551(_new_view)
                                        3    0.000    0.000  C:\Users\Jason\Ana

# Code Profiling Strategy

Generally, it is important to focus your effort on speeding up the 1 or 2 slowest functions. If you speed up something that takes 50% of the runtime by a factor of 100, you speed up the code by a factor of 2. Even if a piece of code takes 10% of the runtime, a factor of 2 improvement (which by itself is often hard to get) only gives you a 5% improvement in the end. **Focus your effort on optimizing the runtime of the one or two functions that take the most amount of time.**

For all else, it is generally more important to optimize code readibility over runtime. Code that is more readible is less prone to bugs and eaiser to maintain. If a piece of code takes only 0.0001% of the runtime, any amount of speed up is not worth making the code hard to read (from a person-hour perspective). Many times, it is difficult to gain more than a factor of 2 speed up in runtime. Definitely think about the potential gain in speed-up versus how long it takes to write and maintian the code.